In [36]:
import os
import pandas as pd
from sqlalchemy import create_engine

In [37]:
POSTGRES_HOST = os.environ.get("POSTGRES_HOST")
POSTGRES_DB = os.environ.get("POSTGRES_DB")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DB}"

In [38]:
engine = create_engine(POSTGRES_URL)

In [45]:
def get_view_definition(view_name):
    query = """
    SELECT view_definition
    FROM information_schema.views
    WHERE table_name = %s
    """

    try:
        with engine.connect() as connection:
            result = pd.read_sql_query(
                query, 
                connection, 
                params=(view_name,)
            )
            if not result.empty:
                return result.iloc[0]['view_definition']
            else:
                return f"View '{view_name}' not found"
    except Exception as e:
        return f"Error retrieving view definition: {e}"


In [46]:
def get_view_sample_data(view_name):
    query = f"SELECT * FROM {view_name} LIMIT 10"

    try:
        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
            if not result.empty:
                return result
            else:
                return f"View '{view_name}' not found"
    except Exception as e:
        return f"Error querying view: {e}"

In [61]:
q1_query = get_view_definition('vw_question_1')
print('\nQuestion 1 Query:\n')
print(q1_query)
print('\n')


Question 1 Query:

 WITH member_sections_2022 AS (
         SELECT DISTINCT t.acct_id,
            t.section_name
           FROM tickets t
          WHERE (((t.plan_event_name)::text ~~ '22FS%'::text) AND (t.ticket_status = ANY (ARRAY['A'::bpchar, 'Active'::bpchar])))
        )
 SELECT acct_id,
    array_agg(section_name) AS sections
   FROM member_sections_2022
  GROUP BY acct_id;




In [53]:
q1_sample_data = get_view_sample_data('vw_question_1')
print('\nQuestion 1 Sample Data:\n')
print(q1_sample_data)
print('\n')


Question 1 Sample Data:

   acct_id      sections
0    19522      [FLDCLD]
1    19231      [FLDCLE]
2    18571       [SUPP1]
3    10276      [FNDCLB]
4    20950         [107]
5    10201      [FLCDWC]
6    36037  [225, SUPP5]
7    18181       [SUPP2]
8    35532         [211]
9    50967         [109]




In [54]:
q2_query = get_view_definition('vw_question_2')
print('\nQuestion 2 Query:\n')
print(q2_query)
print('\n')


Question 2 Query:

 WITH acct_ticket_spend AS (
         SELECT t.acct_id,
            sum(
                CASE
                    WHEN (EXTRACT(year FROM e.event_date) = (2021)::numeric) THEN t.purchase_price
                    ELSE (0)::numeric
                END) AS ticket_spend_2021,
            sum(
                CASE
                    WHEN (EXTRACT(year FROM e.event_date) = (2022)::numeric) THEN t.purchase_price
                    ELSE (0)::numeric
                END) AS ticket_spend_2022
           FROM (tickets t
             LEFT JOIN events e ON ((t.event_id = e.event_id)))
          WHERE ((t.ticket_status = ANY (ARRAY['A'::bpchar, 'Active'::bpchar])) AND (t.purchase_price IS NOT NULL))
          GROUP BY t.acct_id
        ), acct_merch_spend AS (
         SELECT m.acct_id,
            COALESCE(sum(m.line_total), (0)::numeric) AS merch_spend
           FROM merchandise m
          GROUP BY m.acct_id
        )
 SELECT COALESCE(ts.acct_id, ms.acct_id) AS acct_id,
  

In [55]:
q2_sample_data = get_view_sample_data('vw_question_2')
print('\nQuestion 2 Sample Data:\n')
print(q2_sample_data)
print('\n')


Question 2 Sample Data:

   acct_id  ticket_spend_2021  ticket_spend_2022  merchandise_spend
0       -2             114.00             454.50               0.00
1      503               0.00               0.00               0.00
2    10001            1349.26             343.22               0.00
3    10003            7229.22            6572.16               0.00
4    10007            5667.08            7370.16               0.00
5    10009            2560.78            3380.04              40.49
6    10011            2698.52            3653.12               0.00
7    10012            2960.78            3982.10               0.00
8    10013            5201.52               0.00               0.00
9    10014            2295.76            3108.04               0.00




In [56]:
q3_query = get_view_definition('vw_question_3')
print('\nQuestion 3 Query:\n')
print(q3_query)
print('\n')


Question 3 Query:

 WITH earliest_ticket_purchases AS (
         SELECT t.acct_id,
            min(e.event_date) AS first_purchase
           FROM (tickets t
             JOIN events e ON ((t.event_id = e.event_id)))
          WHERE (t.ticket_status = ANY (ARRAY['A'::bpchar, 'Active'::bpchar]))
          GROUP BY t.acct_id
        ), earliest_merch_purchases AS (
         SELECT merchandise.acct_id,
            min(merchandise.order_date) AS first_purchase
           FROM merchandise
          GROUP BY merchandise.acct_id
        )
 SELECT acct_id,
    min(first_purchase) AS earliest_purchase_date
   FROM ( SELECT earliest_ticket_purchases.acct_id,
            earliest_ticket_purchases.first_purchase
           FROM earliest_ticket_purchases
        UNION ALL
         SELECT earliest_merch_purchases.acct_id,
            earliest_merch_purchases.first_purchase
           FROM earliest_merch_purchases) purchases
  GROUP BY acct_id;




In [57]:
q3_sample_data = get_view_sample_data('vw_question_3')
print('\nQuestion 3 Sample Data:\n')
print(q3_sample_data)
print('\n')


Question 3 Sample Data:

    acct_id earliest_purchase_date
0     75262             2021-06-19
1  10502369             2022-03-03
2     18803             2021-06-19
3    156152             2021-06-23
4   9452433             2021-08-04
5  10576058             2022-03-06
6     31789             2021-06-19
7   2620583             2021-05-15
8  10438653             2022-02-26
9     20549             2021-06-19




In [58]:
q4_query = get_view_definition('vw_question_4')
print('\nQuestion 4 Query:\n')
print(q4_query)
print('\n')


Question 4 Query:

 WITH last_match AS (
         SELECT events.event_id
           FROM events
          WHERE (((events.season_name)::text = '2022 LAFC Season'::text) AND ((events.minor_category)::text = 'MLS SOCCER'::text))
          ORDER BY events.event_date DESC
         LIMIT 1
        ), sections AS (
         SELECT m.section_id,
            sum(COALESCE((m.num_seats)::integer, 0)) AS capacity
           FROM manifests m
          GROUP BY m.section_id
        )
 SELECT s.section_id,
    ((count(t.ticket_id))::double precision / (NULLIF(s.capacity, 0))::double precision) AS sell_through_rate
   FROM (sections s
     LEFT JOIN tickets t ON (((t.section_id = s.section_id) AND (t.event_id = ( SELECT last_match.event_id
           FROM last_match)) AND (t.ticket_status = ANY (ARRAY['A'::bpchar, 'Active'::bpchar])))))
  GROUP BY s.section_id, s.capacity;




In [59]:
q4_sample_data = get_view_sample_data('vw_question_4')
print('\nQuestion 4 Sample Data:\n')
print(q4_sample_data)
print('\n')


Question 4 Sample Data:

   section_id  sell_through_rate
0        1267           0.952727
1        1268           0.714286
2        1269           0.833333
3        1270           0.833333
4        1271           1.000000
5        1272           1.000000
6        1273           0.333333
7        1274           0.656475
8        1275           0.181818
9        1276           0.989035


